<img src = "./resources/images/header_banner_3.jpeg" alt = "Encabezado MLDS" width = "100%">  </img>

# **Preparación de los datos**
---

## **0. Integrantes del equipo de trabajo**
---

<table><thead>
  <tr>
    <th>#</th>
    <th>Integrante</th>
    <th>Documento de identidad</th>
  </tr></thead>
<tbody>
  <tr>
    <td>1</td>
    <td>Laura Alejandra Díaz López</td>
    <td>1010018062</td>
  </tr>
  <tr>
    <td>2</td>
    <td>Diego Alejandro Feliciano Ramos</td>
    <td>1024586904</td>
  </tr>
  <tr>
    <td>3</td>
    <td>Geraldine Gracia Ruiz</td>
    <td>1032488268</td>
  </tr>
</tbody>
</table>

## **1. Limpieza de los Datos**
---

La elección de las técnicas de preprocesamiento puede diferir en cada conjunto de datos. Recuerde que es posible aplicar, según sea necesario (no necesariamente todas), las técnicas generales que se han explorado en el curso. La elección dependerá del tipo de datos con el que esté trabajando.

A lo largo de esta entrega, busque responder las siguientes preguntas:

- ¿Cuáles fueron los criterios utilizados para identificar y tratar valores atípicos, datos faltantes o cualquier otra anomalía en el conjunto de datos durante el proceso de limpieza?
- ¿Cómo se justificaría la necesidad de cada paso de preprocesamiento en términos de mejora de la calidad de los datos y preparación para el análisis subsiguiente?

A continuación encontrará los puntos a tratar a medida que va realizando la preparación de los datos. En cada punto defina el estado en que se encontraba el dataset, ademas de explicar y justificar las acciones y decisiones que se tomaron.

In [ ]:
# Instalamos los paquetes necesarios para el notebook en caso de trabajar desde Google Colab
!pip install wget pandas ydata-profiling ipywidgets

In [ ]:
import wget, pandas as pd

# Descargar el dataset transformado de la entrega anterior del repositorio publico
url = 'https://github.com/MLDS-UN-ProjectTeam/final-term-project/raw/main/resources/data/Saber_11_2023-2_-_Transformado.csv?download='
downloaded_file_name = 'Saber_11_2023-2_-_Transformado.csv'
wget.download(url = url, out = downloaded_file_name)

# Cargar en el notebook el conjunto de datos con las restricciones conocidas:
# - Delimitador: ¬ 
# - Quoting = 3 => Sin comillas para todas las variables
saber11_dataframe = pd.read_csv(downloaded_file_name, delimiter='¬', quoting=3)

### **1.1. Valores faltantes**
---
Al encontrarnos con valores faltantes en el conjunto de datos, es crucial preguntarse:
1. **¿Cómo afectan estos valores a la integridad y representatividad de la información?**

La ausencia o falta de datos en un conjunto puede impactar negativamente tanto la calidad como la precisión de la información analizada. Esto ocurre de dos maneras principales:
* En cuanto a la calidad de los datos:
    * Los análisis pueden ser imprecisos debido a cálculos erróneos.
    * Se puede perder información valiosa, limitando la utilidad del análisis.
    * Los modelos predictivos pueden fallar o funcionar de manera deficiente.
* Respecto a la precisión de los datos:
    * Pueden surgir sesgos si la falta de datos no es aleatoria.
    * Por ejemplo, en estudios médicos, si los pacientes más graves tienen más datos faltantes, se podría subestimar la severidad de la enfermedad.
    * La capacidad de detectar patrones significativos se reduce al haber menos datos disponibles.
Para abordar estos problemas, existen varias estrategias:
* Eliminar registros incompletos, aunque esto puede resultar en pérdida de información.
* Estimar los valores faltantes, pero esto puede introducir suposiciones inexactas.
* Utilizar algoritmos diseñados para manejar datos incompletos.

2. **¿Cómo se identificaron los valores faltantes en el conjunto de datos?**

In [ ]:
#Para la identificación de valores faltantes usamos pandas y varias de las funciones que nos ofrece su api, aquí será descrito cada método que se usó y la finalidad con la que se usó

# Contar el total de valores faltantes en el DataFrame
total_faltantes = saber11_dataframe.isnull().sum().sum()
print('\nTotal faltantes = ' + str(total_faltantes))

#Cantidad de datos nulos por columna
columnas_con_faltantes = saber11_dataframe.isnull().sum()
columnas_con_faltantes = columnas_con_faltantes[columnas_con_faltantes != 0]
print(columnas_con_faltantes)

# Rellenamos columnas de interés con la media de las ocurrencias
media_estrato_vivienda = saber11_dataframe['FAMI_ESTRATOVIVIENDA_N'].mean()
saber11_dataframe['FAMI_ESTRATOVIVIENDA_N'] = saber11_dataframe['FAMI_ESTRATOVIVIENDA_N'].fillna(media_estrato_vivienda)



3. **¿Cuáles fueron los criterios para decidir si rellenar con valores estimados o eliminar los valores faltantes? En caso que aplique, ¿qué método de relleno se utilizó y por qué se consideró apropiado?**

El criterio que se usa y usará durante esta entrega será el de que sea una variable numérica, no categórica, ya que nos permite ver la distribución de los datos, así como que no tenga una gran cantidad de datos faltantes, estimando esta gran cantidad en más del 10% de los datos ~ > 55100 datos por columna como máximo. lo cuál solo ocurre para una variable de interés (`FAMI_ESTRATOVIVIENDA_N`). 

### **1.2. Valores duplicados**
----
En el análisis del conjunto de datos, se comprobó que no existen valores duplicados. El conjunto de datos no presenta duplicados. La variable *ESTU_CONSECUTIVO*, que corresponde al ID público del inscrito (SB11) fue estudiada y se verificó que la cantidad de valores únicos en esta columna coincide con el total de registros del conjunto de datos. Esto confirma que cada registro representa a un inscrito diferente.





In [4]:
print(f"Cantidad de filas en el conjunto de datos: {saber11_dataframe.shape[0]}")
print(f"Cantidad de valores únicos en la variable ESTU_CONSECUTIVO: {saber11_dataframe['ESTU_CONSECUTIVO'].nunique()}")

Cantidad de filas en el conjunto de datos: 551149
Cantidad de valores únicos en la variable ESTU_CONSECUTIVO: 551149


### **1.3. Valores atípicos**
---
Al abordar valores atípicos, es relevante cuestionarse sobre la naturaleza de estos puntos extremos.
* ¿Son errores de medición o representan información válida pero excepcional?

En nuestro caso representan información valida pero excepcional, esto se ve reflejado particularmente en las variables que han sido transformadas, como en el caso de la cantidad de libros leídos por familia(`FAMI_NUMLIBROS_N`) y la correlación directa con la variable objetivo a estimar (`PUNT_GLOBAL`) 

* ¿Qué criterios o técnicas se utilizaron para identificar los valores atípicos?
* ¿Se aplicaron métodos estadísticos o visuales para detectar los valores atípicos?

Para responder a las anteriores preguntas se usó la técnica descrita en la próxima celda de código, donde nos apoyamos en los conceptos estadísticos de cuantiles y rango intercuartilico con las variables numéricas para hallar los valores atípicos; gracias a esto también llegamos a la conclusión anterior. En la que establecemos que es información válida pero excepcional al analizar los, aproximadamente, 186000 registros, esto es, aproximadamente un 33% del conjunto de datos.

Aquí vemos que la cantidad de outliers en realidad parece ser dada cuando no hay datos disponibles en alguna de las columnas numéricas, tomando como fuente de verdad la cantidad de NaN en el mismo dataframe retornado por la función   

In [5]:
# Función para identificar outliers usando el rango intercuartilico
def identificar_outliers_iqr(df, multiplicador = 1.5):
    outliers = pd.DataFrame()
    
    for col in saber11_dataframe.select_dtypes(include='number').columns:  # Solo columnas numéricas
        q1 = df[col].quantile(0.25)
        q3 = df[col].quantile(0.75)
        rqi = q3 - q1

        limite_inferior = q1 - (multiplicador * rqi)
        limite_superior = q3 + (multiplicador * rqi)
        
        # Filtrar outliers para cada columna
        outliers_col = df[(df[col] < limite_inferior) | (df[col] > limite_superior)]
        
        # Concatenar resultados
        outliers = pd.concat([outliers, outliers_col])
    
    return outliers.drop_duplicates()

# Identificamos outliers en el DataFrame completo
outliers = identificar_outliers_iqr(saber11_dataframe)
print(outliers)

# Identificamos las columnas con outliers
filtro_outliers = outliers.isna().sum() != 0
print(outliers.isnull().sum()[filtro_outliers])


        Unnamed: 0 ESTU_TIPODOCUMENTO ESTU_NACIONALIDAD ESTU_GENERO  \
50376        50376                PPT         VENEZUELA           F   
163284      163284                NES         VENEZUELA           F   
267396      267396                 TI          COLOMBIA           M   
340253      340253                 CC           ECUADOR           M   
372438      372438                 TI          COLOMBIA           F   
...            ...                ...               ...         ...   
551134      551134                 TI          COLOMBIA           M   
551135      551135                 TI          COLOMBIA           M   
551136      551136                 CC          COLOMBIA           F   
551137      551137                 TI          COLOMBIA           M   
551138      551138                 CC          COLOMBIA           M   

       ESTU_FECHANACIMIENTO  PERIODO  ESTU_CONSECUTIVO ESTU_ESTUDIANTE  \
50376            24/02/2004    20234  SB11202340049505      ESTUDIANTE   


* ¿Cuál fue la decisión final sobre cómo tratar los valores atípicos y por qué?


Debido al impacto que tienen estos valores atípicos, afectando al 33% del conjunto de datos, se ha decidido:
* En primer lugar, ampliar la ventana de elección de rangos inferiores y superiores para las variables numéricas para obtener mayor cantidad de datos ya que estas ocurrencias son, en su mayoría, medidas excepcionales sobre la variable tomada
* En segundo lugar, eliminar el resto de ocurrencias con la nueva aproximación tomada. Esto se puede ver en la siguiente celda de código.  

In [6]:
# Identificamos los nuevos outliers en el DataFrame completo
outliers = identificar_outliers_iqr(saber11_dataframe, 2)
print(outliers)

# Identificamos las columnas con outliers
filtro_outliers = outliers.isna().sum() != 0
print(outliers.isnull().sum()[filtro_outliers])

# Convertir filas a tuplas para comparación
tuplas_saber11_dataframe = saber11_dataframe.apply(tuple, axis=1)
tuplas_outliers = outliers.apply(tuple, axis=1)

# Eliminar filas en saber_11_dataframe que son outliers
saber11_dataframe_no_outliers = saber11_dataframe[~tuplas_saber11_dataframe.isin(tuplas_outliers)]
saber11_dataframe_no_outliers.info()

        Unnamed: 0 ESTU_TIPODOCUMENTO ESTU_NACIONALIDAD ESTU_GENERO  \
50376        50376                PPT         VENEZUELA           F   
163284      163284                NES         VENEZUELA           F   
267396      267396                 TI          COLOMBIA           M   
340253      340253                 CC           ECUADOR           M   
372438      372438                 TI          COLOMBIA           F   
...            ...                ...               ...         ...   
551134      551134                 TI          COLOMBIA           M   
551135      551135                 TI          COLOMBIA           M   
551136      551136                 CC          COLOMBIA           F   
551137      551137                 TI          COLOMBIA           M   
551138      551138                 CC          COLOMBIA           M   

       ESTU_FECHANACIMIENTO  PERIODO  ESTU_CONSECUTIVO ESTU_ESTUDIANTE  \
50376            24/02/2004    20234  SB11202340049505      ESTUDIANTE   

### **1.4. Datos Inconsistentes**
---

El conjunto de datos no presenta inconsistencias en su mayoría. Sin embargo, a partir del análisis realizado en la fase anterior, se identificó una inconsistencia en la variable `COLE_SEDE_PRINCIPAL`, que contiene la respuesta a la pregunta: ¿Esta es la sede principal del Establecimiento Educativo?

De acuerdo con la documentación, las opciones de respuesta son: 'S' o 'N'. No obstante, se observaron cuatro valores únicos en esta columna, lo cual se debe a que en algunas filas la respuesta tiene un espacio adicional al final, quedando como 'S ' y 'N '.

<img src = "./resources/images/COLE_SEDE_PRINCIPAL_inconsistencia.jpg" alt = "COLE_SEDE_PRINCIPAL" width = "100%">  </img>

Aunque la variable no es de interés para objetivo del análisis, esta inconsistencia se corregirá aplicando el método `strip()` a los valores de la columna, lo que eliminará el espacio adicional y asegurará la consistencia de los datos.

In [7]:
# Valores únicos en la columna COLE_SEDE_PRINCIPAL original
print(saber11_dataframe_no_outliers['COLE_SEDE_PRINCIPAL'].unique()) 

# Se eliminan espacios adicionales con el método strip()
saber11_dataframe_no_outliers['COLE_SEDE_PRINCIPAL'] = saber11_dataframe_no_outliers['COLE_SEDE_PRINCIPAL'].str.strip()

# Valores únicos en la columna COLE_SEDE_PRINCIPAL después de corregir
print(saber11_dataframe_no_outliers['COLE_SEDE_PRINCIPAL'].unique()) 

['S' 'N' 'S ' 'N ']
['S' 'N']


/var/folders/18/_z1lzmt167s25frbpysj7rqw0000gn/T/ipykernel_95184/223480693.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  saber11_dataframe_no_outliers['COLE_SEDE_PRINCIPAL'] = saber11_dataframe_no_outliers['COLE_SEDE_PRINCIPAL'].str.strip()


### **1.5. Datos corruptos**
---

En la exploración del conjunto de datos, no se evidencian datos corruptos. En este caso, la información de los inscrito, al ser recolectada a través de una encuesta con preguntas directas y opciones de respuesta predefinidas, se limita la existencia de errores de entrada.

### **1.6. Selección de datos**
---
En la selección de datos, reflexione sobre las variables que son realmente relevantes para sus objetivos de análisis.
* ¿Qué criterios se utilizaron para seleccionar los datos relevantes para el análisis?
* ¿Se aplicaron técnicas de muestreo o filtrado para reducir el tamaño del conjunto de datos?
* ¿Cómo se justificó la inclusión o exclusión de ciertas variables en la selección de datos?

Para reducir el tamaño del conjunto de datos el recurso principal fue enfocar el estudio y análisis solamente a Bogotá D.C, en vez de hacerlo a nivel nacional, lo cual está en concordancia con un tipo de muestreo no probabilístico llamado muestreo intencional. Se consideró la amplia población de la ciudad como una buena muestra dado que en la capital convergen todo tipo de estratos socioeconómicos, experiencias de vida y, adicionalmente, muchos jovenes son obligados a desplazarse a ella a lo largo de su adolescencia, por lo que hacia ella confluyen historias de todo el país. 

Ahora, con mayor enfoque hacia la selección de datos: el criterio principal fue el objetivo del estudio, encontrar relaciones importantes entre variables de indicadores socioeconómicos y el resultado del examen de estado. Esta filtración de datos se ha venido haciendo desde la entrega anterior, pues ha sido fácil excluir variables que no proveían información de interés para lo que compete a este análisis. Adicional a esto, al evaluar la calidad de los datos, las variables con un alto porcentaje de valores nulos también decidieron descartarse dado su aporte turbulento al dataset.

In [10]:
from ydata_profiling import ProfileReport
# De acuerdo con el profiling y el trabajo realizado en entregas anteriores se eliminarán columnas que tienen alguna de las siguientes características:
# - Tienen todos sus valores como ocurrencias únicas
# - Son constantes
# - Tienen alta correlación
# - Tienen una gran cantidad (>10%) de datos faltantes
# - No son útiles a nuestro entender para la intención que se tiene con el set de datos
# - Tienen un desbalanceo muy fuerte, donde el 90% o una mayor proporción de las ocurrencias está en una sola categoría/etiqueta para la columna eliminada
# - Se explican gracias a otras (factores socioeconómicos con nivel socioeconómico, por ejemplo) 
columnas_a_eliminar = [
    'ESTU_TIPODOCUMENTO',               #
    'ESTU_NACIONALIDAD',                #
    'ESTU_FECHANACIMIENTO',             #
    'PERIODO',                          #
    'ESTU_CONSECUTIVO',                 #
    'ESTU_ESTUDIANTE',                  #
    'ESTU_PAIS_RESIDE',                 #
    'ESTU_DEPTO_RESIDE',                #
    'ESTU_MCPIO_RESIDE',                #
    'ESTU_COD_RESIDE_DEPTO',            #
    'ESTU_COD_RESIDE_MCPIO',            #
    'ESTU_PRESENTACIONSABADO',          #
    'ESTU_LENGUANATIVA',                #
    'FAMI_ESTRATOVIVIENDA',             #
    'FAMI_TIENEAUTOMOVIL',              #
    'FAMI_TIENECOMPUTADOR',             #
    'FAMI_TIENEHORNOMICROOGAS',         #
    'FAMI_TIENEINTERNET',               #
    'FAMI_TIENELAVADORA',               #
    'FAMI_PERSONASHOGAR',               #
    'FAMI_CUARTOSHOGAR',                #
    'FAMI_TIENESERVICIOTV',             #
    'FAMI_TIENEMOTOCICLETA',            #
    'FAMI_TIENECONSOLAVIDEOJUEGOS',     #
    'FAMI_NUMLIBROS',                   #
    'FAMI_COMELECHEDERIVADOS',          #
    'FAMI_COMECARNEPESCADOHUEVO',       #
    'FAMI_COMECEREALFRUTOSLEGUMBRE',    #
    'FAMI_SITUACIONECONOMICA',          #
    'COLE_AREA_UBICACION',              #
    'COLE_MCPIO_UBICACION',             #
    'COLE_COD_MCPIO_UBICACION',         #
    'COLE_COD_DEPTO_UBICACION',         #
    'COLE_COD_DANE_SEDE',               #
    'COLE_NOMBRE_SEDE',                 #
    'COLE_CALENDARIO',                  #
    'COLE_BILINGUE',                    #
    'COLE_GENERO',                      #
    'COLE_CODIGO_ICFES',                #
    'COLE_COD_DANE_ESTABLECIMIENTO',    #
    'COLE_NOMBRE_ESTABLECIMIENTO',      #
    'COLE_SEDE_PRINCIPAL',              #
    'COLE_DEPTO_UBICACION',             #
    'ESTU_PRIVADO_LIBERTAD',            #
    'ESTU_COD_MCPIO_PRESENTACION',      #
    'ESTU_MCPIO_PRESENTACION',          #
    'ESTU_DEPTO_PRESENTACION',          #
    'ESTU_COD_DEPTO_PRESENTACION',      #
    'PERCENTIL_LECTURA_CRITICA',        #
    'DESEMP_LECTURA_CRITICA',           #
    'PUNT_LECTURA_CRITICA',             #
    'PERCENTIL_MATEMATICAS',            #
    'DESEMP_MATEMATICAS',               #
    'PUNT_MATEMATICAS',                 #
    'PERCENTIL_C_NATURALES',            #
    'DESEMP_C_NATURALES',               #
    'PUNT_C_NATURALES',                 #
    'PERCENTIL_SOCIALES_CIUDADANAS',    #
    'DESEMP_SOCIALES_CIUDADANAS',       #
    'PUNT_SOCIALES_CIUDADANAS',         #
    'PERCENTIL_INGLES',                 #
    'DESEMP_INGLES',                    #
    'PUNT_INGLES',                      #
    'PERCENTIL_GLOBAL',                 #
    'PERCENTIL_ESPECIAL_GLOBAL',        #
    'ESTU_NSE_INDIVIDUAL',              #
    'ESTU_ESTADOINVESTIGACION',         #
    'ESTU_GENERACION',                  #
    'ESTU_GENERO_N',                    #
    'FAMI_PERSONASHOGAR_N',             #
    'FAMI_TIENEINTERNET_N',             #
    'FAMI_TIENECOMPUTADOR_N',           #
    'FAMI_NUMLIBROS_N',                 #
    'ESTU_HORASSEMANATRABAJA_N',        #
    'ESTU_PRIVADO_LIBERTAD_N'           #
]
## Realizamos las consultas para seleccionar solo los estudiantes en Bogotá que también presentaron la prueba en Bogotá
condicion_de_residencia = saber11_dataframe_no_outliers['ESTU_DEPTO_RESIDE'] =='BOGOTÁ'
condicion_de_presentacion = saber11_dataframe_no_outliers['ESTU_DEPTO_PRESENTACION'] =='BOGOTÁ'

# Seleccionamos con la condición anterior la parte de nuestro interés del set de datos, se eliminan las columnas descritas anteriormente y por último se eliminan las filas que quedaron vacías al eliminar outliers y aplicar las condiciones dadas (quedaron como NaN)
saber11_dataframe_nuevo = saber11_dataframe_no_outliers.where(condicion_de_residencia & condicion_de_presentacion).drop(columns=columnas_a_eliminar).drop_duplicates()

# Se genera el _profile_ en un archivo HTML para poder verlo de forma separada y no cargar el notebook con información innecesaria
profile_nuevo = ProfileReport(saber11_dataframe_nuevo)
profile_nuevo.to_file('Saber 11 2023-2-Report - Nuevo.html')

# Se exporta el dataframe a un nuevo archivo para ser trabajado posteriormente:
saber11_dataframe.to_csv(path_or_buf='resources/data/Saber_11_2023-2_-_Preprocesado.csv', sep='¬', na_rep='')

Matplotlib is building the font cache; this may take a moment.


Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

Export report to file:   0%|          | 0/1 [00:00<?, ?it/s]

## **Créditos**
* **Profesor:** [Felipe Restrepo Calle](https://dis.unal.edu.co/~ferestrepoca/)
* **Asistentes docentes:**
    - [Juan Sebastián Lara Ramírez](https://www.linkedin.com/in/juan-sebastian-lara-ramirez-43570a214/).
* **Diseño de imágenes:**
    - [Rosa Alejandra Superlano Esquibel](mailto:rsuperlano@unal.edu.co).
* **Coordinador de virtualización:**
    - [Edder Hernández Forero](https://www.linkedin.com/in/edder-hernandez-forero-28aa8b207/).
    
**Universidad Nacional de Colombia** - *Facultad de Ingeniería*